# Day 1 - Sparse grids

## Homework 1 - Exercise 1

Author: Martina Fraschini

In [8]:
# import packages
import TasmanianSG
import numpy as np
import math
from random import uniform

In [9]:
# define function for interpolation with non-adaptive sparse grids
def nonadaptive(dd, out, lev, basis, cdd):
    # create an empty grid instance
    grid = TasmanianSG.TasmanianSparseGrid()
    
    # construct sparse grid
    grid.makeLocalPolynomialGrid(dd, out, lev, basis, "localp")
    aPoints = grid.getPoints()
    iNumP1 = aPoints.shape[0]
    aVals = np.empty([iNumP1, out])
    for iI in range(iNumP1):
        res = 2 * math.pi * w1
        for iJ in range(dd):
            res += cdd[iJ] * aPoints[iI][iJ]
        aVals[iI] = math.cos(res)
    grid.loadNeededPoints(aVals)
    
    return grid, iNumP1


In [10]:
# define function for interpolation with adaptive sparse grids
def adaptive():




IndentationError: expected an indented block (<ipython-input-10-91b05498eb53>, line 4)

In [11]:
# define function to compute max error
def maxerr(grid, aPnts, aTres):
    aRes = grid.evaluateBatch(aPnts)
    err = max(np.fabs(aRes[:,0] - aTres))
    return err

In [ ]:
# generate plots #points - error
def plotpterr():




In [12]:
# define parameters
dd = 2    # dimension
NN = 1000    # number of test points
w1 = uniform(-1.0, 1.0)    # construction of w1
cdd = np.empty(dd)    # construction of vector of c values
for iJ in range(dd):
    cdd[iJ] = uniform(-1.0, 1.0)

out = 1    # number of outputs for grid
lev = 5    # number of levels for grid
basis = 1    #1= linear basis functions

# 1000 "dim"-dimensional sample points
aPnts = np.empty([NN, dd])
for iI in range(NN):
    for iJ in range(dd):
        aPnts[iI][iJ] = uniform(-1.0, 1.0)

# values of function at sample points
aTres = np.empty([NN,])
for iI in range(NN):
    res = 2 * math.pi * w1
    for iJ in range(dd):
        res += cdd[iJ] * aPnts[iI][iJ]
    aTres[iI] = math.cos(res)


grid, iNumP1 = nonadaptive(dd, out, lev, basis, cdd)
err = maxerr(grid, aPnts, aTres)



OSError: ../../TasmanianSparseGrids/libtasmaniansparsegrid.so: cannot open shared object file: No such file or directory